In [1]:
import pandas as pd
import time
from gensim import corpora, models, similarities
from Models.PhygeVariables import PhyVariables
from Storage import Storage

In [2]:
storage = Storage()
test_case = storage.load_test_case(1)
test_case.setup()

df = pd.read_csv(test_case.path + '/' + PhyVariables.valuesFileKey)
query_normalize = str(test_case.queries[0]).split()

In [3]:
documents = []
for columns in df.columns:
    documents.append(df[columns].dropna().tolist())
dct = corpora.Dictionary(documents)
vec_bow_query = dct.doc2bow(query_normalize)
# dct.save(path + '/deerwester.dict')
corpus = [dct.doc2bow(doc) for doc in documents]
# corpora.MmCorpus.serialize(path + '/deerwester.mm', corpus)  # store to disk, for later use

In [4]:
start_time_LSI = time.time()
lsi = models.LsiModel(corpus, id2word=dct, num_topics=100)
print('LSI model:')
print('Learning time:', round((time.time() - start_time_LSI), 3), 's')

LSI model:
Learning time: 1.058 s


In [5]:
start_time_LSI = time.time()
vec_lsi = lsi[vec_bow_query]  # convert the query to LSI space
index = similarities.MatrixSimilarity(lsi[corpus])
sims = index[vec_lsi]  # perform a similarity query against the corpus
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print('answer', sims[0:5])  # print sorted (document number, similarity score) 2-tuples
print('answer time:', round((time.time() - start_time_LSI), 3), 's')

answer [(18, 0.81367904), (71, 0.53680503), (76, 0.51764625), (70, 0.5085629), (74, 0.37062845)]
answer time: 0.07 s


In [6]:
start_time_LDA = time.time()
lda = models.ldamodel.LdaModel(corpus, id2word=dct, num_topics=100, passes=20)
print('\nLDA model:')
print('Learning time:', round((time.time() - start_time_LDA), 3), 's')
start_time_LDA = time.time()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/models/ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)



LDA model:
Learning time: 55.623 s


In [7]:
vec_lda = lda[vec_bow_query]
index = similarities.MatrixSimilarity(lda[corpus])
sims = index[vec_lda]
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print('answer', sims[0:5])  # print sorted (document number, similarity score) 2-tuples
print('time:', round((time.time() - start_time_LDA), 3), 's')

answer [(18, 0.9457527), (17, 0.18289894), (75, 0.17822975), (73, 0.16977227), (74, 0.16500413)]
time: 1.638 s
